# Test

In [ ]:
import molsysmt as msm

In [ ]:
msm.get_form('traj.msmh5')

In [ ]:
aa = msm.convert('traj.msmh5', 'molsysmt.MSMH5FileHandler')

In [ ]:
output = aa.file['topology']['atoms']['name'][:].astype('int64')

In [ ]:
output

In [ ]:
msm.get('traj.msmh5', element='system', n_structures=True)

In [1]:
molsys = msm.convert('4POC')
molsys = msm.extract(molsys, selection='molecule_type=="protein"')
molsys = msm.build.add_missing_hydrogens(molsys)
molsys = msm.build.solvate(molsys)
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
topology = msm.convert(molsys, 'openmm.Topology')
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
platform = mm.Platform.getPlatformByName("CPU")
simulation = app.Simulation(topology, system, integrator, platform)
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter
from molsysmt.thirds.openmm.reporters import MSMH5Reporter
dict_reporter = MolSysMTTrajectoryDictReporter(10)
simulation.reporters.append(dict_reporter)
msmh5_reporter = MSMH5Reporter('test.msmh5', 10, selection='all', steps=100,
            topology=True, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True,
            compression='gzip', compression_opts=4,
            int_precision='single', float_precision='single',
            syntax='MolSysMT')
simulation.reporters.append(msmh5_reporter)
simulation.step(100)
traj = dict_reporter.finalize()
msmh5_reporter.close()

NameError: name 'msm' is not defined